In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import string

class SimpleRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(SimpleRNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x, hidden):
        batch_size = x.size(0)
        embeds = self.embedding(x) 
        out, hidden = self.rnn(embeds, hidden)
        out = self.fc(out)  # (batch_size, sequence_length, vocab_size)
        return out, hidden
    
    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_dim)

In [ ]:
class TextDataset(Dataset):
    def __init__(self, text, sequence_length):
        self.text = text
        self.sequence_length = sequence_length
        
        # Create vocabulary
        words = text.split()
        self.vocab = ['<PAD>'] + list(set(words))
        self.word_to_idx = {word: idx for idx, word in enumerate(self.vocab)}
        self.idx_to_word = {idx: word for word, idx in self.word_to_idx.items()}
        
        # Create sequences
        self.sequences = []
        self.targets = []
        
        for i in range(len(words) - sequence_length):
            seq = words[i:i + sequence_length]
            target = words[i + sequence_length]
            seq_idx = [self.word_to_idx[word] for word in seq]
            target_idx = self.word_to_idx[target]
            self.sequences.append(seq_idx)
            self.targets.append(target_idx)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return (torch.tensor(self.sequences[idx]), 
                torch.tensor(self.targets[idx]))
    
    def vocab_size(self):
        return len(self.vocab)

In [ ]:
def train_model(model, train_loader, num_epochs, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    # YOUR CODE HERE
    

In [ ]:
# Example usage
def main():
    # Sample text
    text = """I love learning deep learning in Stats 507."""
    
    # Hyperparameters
    sequence_length = 3
    embedding_dim = 64
    hidden_dim = 128
    batch_size = 2
    num_epochs = 10
    
    # Create dataset
    dataset = TextDataset(text, sequence_length)
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Initialize model
    model = SimpleRNN(dataset.vocab_size(), embedding_dim, hidden_dim)
    
    # Train the model
    train_model(model, train_loader, num_epochs)
    
    return model, dataset

if __name__ == "__main__":
    model, dataset = main()